# TINKING

## 在实际工作中，FM和MF哪个应用的更多，为什么(1、能简要说明FM和MF的区别 2、能简要说明FM在推荐系统，以及应用场景中的作用)

MF主要采用svd方式或者asl方式对矩阵进行分解，它只能分解成为两个矩阵和一个特征矩阵的乘积，分别是user矩阵和item矩阵与特征矩阵，通过交替二乘 法或者梯度向量来得到这三个矩阵。
FM是从线性模型中通过增加二阶交叉项来拟合特征之间的交叉，可以分解高阶特征，是一种更加泛化的矩阵分解方式，可以将MF模型看作是FM模型的特例。

FM在推荐系统中可以挖掘出更多高纬度特征，通过简单纬度特征交叉获得更加复杂特征，如通过中国、年龄、女性 可以挖掘出中国大妈这种特征群体从而提升模型效果。通过这种特征可以用来评估站内的CTR和CVR，即一个用户对一个商品的潜在点击率和点击后的转化率。

## FFM与FM有哪些区别？(能简要说明区别)


FM：是解决稀疏数据下的特征组合问题，具有线性的计算复杂度；（矩阵分解方式处理参数，不仅能减少参数数量，还能处理由于稀疏性带来的参数不好训练的问题）一般的线性模型压根没有考虑特征间的关联(组合)。为了表述特征间的相关性，我们采用多项式模型。观察大量的样本数据可以发现，某些特征经过关联之后，与label之间的相关性就会提高

FFM：通过引入field的概念，FFM把相同性质的特征归于同一个field，同一个categorical特征经过One-Hot编码生成的数值特征都可以放到同一个field，包括用户性别、职业、品类偏好等。在FFM中，每一维特征 xi，针对其它特征的每一种field fj，都会学习一个隐向量 vi,fj。因此，隐向量不仅与特征相关，也与field相关，当filed=1时候，FFM就退化为FM了

FFM的计算时间复杂度比FM更大，但是也更加精确



## DeepFM相比于FM解决了哪些问题，原理是怎样的(1、能说明DeepFM相比于FM有哪些改进的地方2、能说明DeepFM的原理，FM+DNN模型)

FM通过对于每一位特征的隐变量内积来提取特征组合，最后的结果也不错，虽然理论上FM可以对高阶特征组合进行建模，但实际上因为计算复杂度原因，一般都只用到了二阶特征组合。对于高阶特征组合来说，一般会使用多层神经网络DNN。

DeepFM目的是同时学习低阶和高阶的特征交叉，主要由FM和DNN两部分组成，底部共享同样的输入。



## Surprise工具中的baseline算法原理是怎样的？BaselineOnly和KNNBaseline有什么区别？(1、能简要说明baseline的原理 2、能简要说明BaselineOnly和KNNBaseline的区别)
 

Baseline算法是基于统计的基准预测线打分，bui 预测值 bu 用户对整体的偏差 bi 商品对整体的偏差，它是通过最小二乘法来计算的，先固定bu来优化bi，然后在固定bi，优化bu（这两个过程是可以并行的）
KNNBaseline 是考虑基线评级的协同过滤，利用行为的相似度计算用户的相似度
BaselineOnly 是以评分基数为基础的打分，它综合了用户和item的历史打分情况

 ## 基于邻域的协同过滤都有哪些算法，请简述原理 (1、能说出两种不同的基于邻域的协同过滤的算法  2、这些算法之间的区别和作用)
 

KNNBasic 是基础的协同过滤，通过它可以计算用户与用户之间的相似性或者物品与物品之间的相似性

KNNWithMeans基本的假设是用户和物品的评分有高低，考虑了每个用户打分均值或者每个item打分的均值，去除参考用户打分整体偏高和偏低的影响

KNNBaseline是对KNNWithMeans的改进，采用baseline均值来替代均值


# Action

## 使用libfm工具对movielens进行评分预测，采用SGD优化算法

In [9]:
# perl ./code/libfm/triple_format_to_libfm.pl -in ./titanic/test.csv -target 1 -delete_column 0 -separator "::"
!pip install -U deepctr

Requirement already up-to-date: deepctr in /Users/hui/opt/anaconda3/lib/python3.8/site-packages (0.8.2)


## 使用DeepFM对movielens进行评分预测

In [17]:
#!pip install deepctr
# !pip install tensorflow
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from deepctr.models import DeepFM
from deepctr.inputs import SparseFeat,get_feature_names

#数据加载

data = pd.read_csv("./code/deepfm/movielens_sample.txt")
sparse_features = ["movie_id", "user_id", "gender", "age", "occupation", "zip"]
target = ['rating']

# 对特征标签进行编码
for feature in sparse_features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature])
# 计算每个特征中的 不同特征值的个数
fixlen_feature_columns = [SparseFeat(feature, data[feature].nunique()) for feature in sparse_features]
print(fixlen_feature_columns)
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

# 将数据集切分成训练集和测试集
train, test = train_test_split(data, test_size=0.2)
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

# 使用DeepFM进行训练
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='regression')
model.compile("adam", "mse", metrics=['mse'], )
history = model.fit(train_model_input, train[target].values, batch_size=256, epochs=1, verbose=True, validation_split=0.2, )
# 使用DeepFM进行预测
pred_ans = model.predict(test_model_input, batch_size=256)
# 输出RMSE或MSE
mse = round(mean_squared_error(test[target].values, pred_ans), 4)
rmse = mse ** 0.5
print("test RMSE", rmse)

ImportError: cannot import name 'SparseFeat' from 'deepctr.inputs' (/Users/hui/opt/anaconda3/lib/python3.8/site-packages/deepctr/inputs.py)

## 使用基于邻域的协同过滤（KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline中的任意一种）对MovieLens数据集进行协同过滤，采用k折交叉验证(k=3)，输出每次计算的RMSE, MAE

In [16]:
## KNNWithMeans 方式
from surprise import KNNWithMeans
from surprise import Dataset, Reader
import pandas as pd
import numpy as np
from surprise import accuracy
from surprise.model_selection import KFold

# 数据读取
reader = Reader(line_format='user item rating timestamp', sep=',', skip_lines=1)
data = Dataset.load_from_file('./code/knn_cf/ratings.csv', reader=reader)

# ItemCF 计算得分
# 取最相似的用户计算时，只取最相似的k个
algo = KNNWithMeans(k=50, sim_options={'user_based': False, 'verbose': 'True'})
resme=[]
mae=[]
kf = KFold(n_splits=3)
for trainset, testset in kf.split(data):
    # 训练并预测
    algo.fit(trainset)
    predictions = algo.test(testset)
    # 计算RMSE
    resme.append(accuracy.rmse(predictions, verbose=False))
    mae.append(accuracy.mae(predictions, verbose=False))
    uid = str(196)
    iid = str(302)
    pred = algo.predict(uid, iid)

resme.append(np.mean(resme))
mae.append(np.mean(mae))
pd=pd.DataFrame(np.array([resme,mae]),index=['rmse','mae'],columns=['第一次','第二次','第三次','均值'])
print(pd) 


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
           第一次       第二次       第三次        均值
rmse  0.858293  0.855608  0.855494  0.856465
mae   0.655689  0.654485  0.654445  0.654873
